In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
from datetime import datetime
import random
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input as efficientnetv2_preprocess

from tensorflow.keras.applications import (
    MobileNetV3Small,
    MobileNetV3Large,
    EfficientNetV2B0,
    EfficientNetV2B1,
    EfficientNetV2B2,
    EfficientNetV2B3,
    EfficientNetV2S,
    EfficientNetV2M,
    EfficientNetV2L
)

from efficientNet.efficientnet_v2_modifikasi import (
    EfficientNetV2B0Modifikasi,
    EfficientNetV2B1Modifikasi,
    EfficientNetV2B2Modifikasi,
    EfficientNetV2B3Modifikasi,
)

2024-12-25 14:21:01.971224: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735111262.013732   29226 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735111262.026655   29226 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-25 14:21:02.196132: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Hyperparameter settings
batch_size = 32
epochs_size = 30
optimizer = "Adam"
dropout_rate = 0.2
learning_rate = 0.001
trainable_percentage = 0.3
trainable_percentage_modifikasi = 1

# Dataset directories
dataset_path = "datasets/"
train_dir = os.path.join(dataset_path, "training")
val_dir = os.path.join(dataset_path, "validation")
test_dir = os.path.join(dataset_path, "testing")

# Save paths
save_model_path = "results/best_model/kFold/"
result_csv_path = "results/csv/kFold/"
image_path = "results/images/kFold/"

input_shapes = {
    "MobileNetV3Small": (224, 224),
    "MobileNetV3Large": (224, 224),
    "EfficientNetV2B0": (224, 224),
    "EfficientNetV2B1": (240, 240),
    "EfficientNetV2B2": (260, 260),
    "EfficientNetV2B3": (300, 300),
    "EfficientNetV2B0Modifikasi": (224, 224),
    "EfficientNetV2B1Modifikasi": (240, 240),
    "EfficientNetV2B2Modifikasi": (260, 260),
    "EfficientNetV2B3Modifikasi": (300, 300),
}

# Random seed
random.seed(42)

# Select model
selected_model = "EfficientNetV2B0"  # Ganti model sesuai kebutuhan
img_height, img_width = input_shapes[selected_model]

def augmentation_generator_ENV2(img_height, img_width):
    train_datagen = ImageDataGenerator(
        preprocessing_function=efficientnetv2_preprocess,
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    val_test_datagen = ImageDataGenerator(preprocessing_function=efficientnetv2_preprocess)
    return train_datagen, val_test_datagen

def prepare_dataset(train_dir):
    """Membuat daftar semua file dan label di direktori pelatihan."""
    classes = os.listdir(train_dir)
    filepaths = []
    labels = []

    for label, cls in enumerate(classes):
        cls_path = os.path.join(train_dir, cls)
        for fname in os.listdir(cls_path):
            filepaths.append(os.path.join(cls_path, fname))
            labels.append(label)

    return np.array(filepaths), np.array(labels)

def build_model(base_model, num_classes, dropout_rate):
    """Membangun model dengan arsitektur kustom."""
    return models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(128, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(num_classes, activation='softmax')
    ])

def compile_model(model, learning_rate):
    """Mengompilasi model dengan optimizer dan loss function."""
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

def calculate_mean_std(results):
    """Menghitung mean dan std dari hasil fold."""
    mean_accuracy = np.mean(results['accuracy'])
    std_accuracy = np.std(results['accuracy'])
    mean_loss = np.mean(results['loss'])
    std_loss = np.std(results['loss'])
    return {
        'mean_accuracy': mean_accuracy,
        'std_accuracy': std_accuracy,
        'mean_loss': mean_loss,
        'std_loss': std_loss
    }

filepaths, labels = prepare_dataset(train_dir)

# Membagi dataset menggunakan KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results = {'fold': [], 'accuracy': [], 'loss': []}

for fold, (train_index, val_index) in enumerate(kf.split(filepaths)):
    print(f"Fold {fold + 1}")

    train_files, val_files = filepaths[train_index], filepaths[val_index]
    train_labels, val_labels = labels[train_index].astype(str), labels[val_index].astype(str)

    # Data augmentation
    train_datagen, val_test_datagen = augmentation_generator_ENV2(img_height, img_width)

    train_generator = train_datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': train_files, 'class': train_labels}),
        x_col='filename',
        y_col='class',
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )

    val_generator = val_test_datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': val_files, 'class': val_labels}),
        x_col='filename',
        y_col='class',
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    # Load Base Model
    base_model = layers.Input(shape=(img_height, img_width, 3))

    # Build and Compile Model
    model = build_model(base_model, len(train_generator.class_indices), dropout_rate)
    compile_model(model, learning_rate)

    # Train Model
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs_size
    )

    # Evaluate Model
    eval_result = model.evaluate(val_generator)
    results['fold'].append(fold + 1)
    results['accuracy'].append(eval_result[1])
    results['loss'].append(eval_result[0])

# Calculate Mean and Std
metrics = calculate_mean_std(results)
print("Mean Accuracy:", metrics['mean_accuracy'])
print("Std Accuracy:", metrics['std_accuracy'])
print("Mean Loss:", metrics['mean_loss'])
print("Std Loss:", metrics['std_loss'])

# Save results to CSV
results_df = pd.DataFrame(results)
results_df['mean_accuracy'] = metrics['mean_accuracy']
results_df['std_accuracy'] = metrics['std_accuracy']
results_df['mean_loss'] = metrics['mean_loss']
results_df['std_loss'] = metrics['std_loss']
results_df.to_csv(os.path.join(result_csv_path, 'kfold_results.csv'), index=False)


Fold 1
Found 745 validated image filenames belonging to 3 classes.
Found 187 validated image filenames belonging to 3 classes.


I0000 00:00:1735111608.600521   29226 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4080 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5
/home/abiyamf/miniconda3/envs/projectAI/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


I0000 00:00:1735111614.808639   29423 service.cc:148] XLA service 0x7fdbfc003da0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1735111614.808788   29423 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2024-12-25 14:26:54.851306: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1735111615.016862   29423 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/24 ━━━━━━━━━━━━━━━━━━━━ 1:13 3s/step - accuracy: 0.1875 - loss: 16.1020

...

Epoch 1/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 13s 395ms/step - accuracy: 0.3309 - loss: 11.3595 - val_accuracy: 0.5376 - val_loss: 1.0531
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 351ms/step - accuracy: 0.3777 - loss: 5.7546 - val_accuracy: 0.4462 - val_loss: 2.1540
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 326ms/step - accuracy: 0.3663 - loss: 3.7486 - val_accuracy: 0.5108 - val_loss: 1.0055
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 290ms/step - accuracy: 0.4045 - loss: 2.0403 - val_accuracy: 0.4086 - val_loss: 1.0316
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 303ms/step - accuracy: 0.3712 - loss: 1.4968 - val_accuracy: 0.3978 - val_loss: 1.0782
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 307ms/step - accuracy: 0.3736 - loss: 1.2301 - val_accuracy: 0.4301 - val_loss: 1.0884
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 305ms/step - accuracy: 0.3890 - loss: 1.1919 - val_accuracy: 0.4409 - val_loss: 1.0805
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 293ms/step - accuracy: 0.4160 - loss: 1.1209 - val_accuracy: 

/home/abiyamf/miniconda3/envs/projectAI/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 11s 349ms/step - accuracy: 0.3680 - loss: 8.9192 - val_accuracy: 0.6237 - val_loss: 1.7614
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 302ms/step - accuracy: 0.3666 - loss: 4.9072 - val_accuracy: 0.5968 - val_loss: 1.3836
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 309ms/step - accuracy: 0.4535 - loss: 2.4014 - val_accuracy: 0.5323 - val_loss: 1.0253
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 323ms/step - accuracy: 0.4346 - loss: 1.6500 - val_accuracy: 0.4247 - val_loss: 1.0537
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 315ms/step - accuracy: 0.4225 - loss: 1.2456 - val_accuracy: 0.3280 - val_loss: 1.1269
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 308ms/step - accuracy: 0.4152 - loss: 1.1680 - val_accuracy: 0.6129 - val_loss: 1.0446
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 306ms/step - accuracy: 0.4575 - loss: 1.1274 - val_accuracy: 0.6075 - val_loss: 1.0277
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 308ms/step - accuracy: 0.4462 - loss: 1.0408 - val_accuracy: 0

/home/abiyamf/miniconda3/envs/projectAI/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 11s 353ms/step - accuracy: 0.3762 - loss: 13.6821 - val_accuracy: 0.3172 - val_loss: 2.7883
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 381ms/step - accuracy: 0.3881 - loss: 5.0298 - val_accuracy: 0.4785 - val_loss: 1.2642
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 343ms/step - accuracy: 0.4213 - loss: 3.0113 - val_accuracy: 0.3333 - val_loss: 1.2111
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 307ms/step - accuracy: 0.4176 - loss: 1.9688 - val_accuracy: 0.4624 - val_loss: 1.0210
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 319ms/step - accuracy: 0.4039 - loss: 1.3986 - val_accuracy: 0.5161 - val_loss: 1.0483
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 325ms/step - accuracy: 0.4436 - loss: 1.1154 - val_accuracy: 0.4301 - val_loss: 1.0563
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 325ms/step - accuracy: 0.3961 - loss: 1.1915 - val_accuracy: 0.4892 - val_loss: 1.0539
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 348ms/step - accuracy: 0.4321 - loss: 1.1553 - val_accuracy: 